In [ ]:
%load_ext taegis_magic

from taegis_magic.pandas.alerts import inflate_raw_events

In [ ]:
%%taegis alerts search --assign alerts_df
FROM alert
WHERE
    status = 'OPEN' AND
    metadata.severity > 0.2 AND
    investigation_ids IS NULL
EARLIEST=-3d

In [ ]:
alerts_df = alerts_df.pipe(inflate_raw_events)

In [ ]:
alerts_df[["metadata.title", "metadata.severity", "event_data.user_name"]]

In [ ]:
alerts_evidence = alerts_df[alerts_df["metadata.title"]=="Suspicious AWS Account Enumeration"].head()

In [ ]:
alerts_evidence["event_data.user_name"].unique()[0]

In [ ]:
%taegis investigations evidence stage alerts alerts_evidence

In [ ]:
%%taegis events search --assign events_evidence
FROM cloudaudit 
WHERE 
    user_name = 'jupiter-one-service-role/jupiterone-184e20ef-e863-4614-8ce5-1c1ec18fe8ed' 
EARLIEST=-3d | head 2

In [ ]:
events_evidence = events_df.head()

In [ ]:
%taegis investigations evidence stage events events_evidence

In [ ]:
%taegis investigations evidence show

In [ ]:
%taegis investigations create \
--title "Micah Magic Test" \
--key-findings "kf.md" \
--priority LOW \
--type SECURITY_INVESTIGATION \
--status DRAFT \
--assignee-id @customer \
--dry-run

In [ ]:
from taegis_sdk_python import GraphQLService
service = GraphQLService()
user = service.users.query.current_tdruser()
assignee_id = user.user_id
title = f"{user.given_name} Test"

In [ ]:
%%writefile kf.md
This is a test.

In [ ]:
%taegis investigations create \
--title $title \
--key-findings "kf.md" \
--priority LOW \
--type SECURITY_INVESTIGATION \
--status DRAFT \
--assignee-id $assignee_id